<a href="https://colab.research.google.com/github/u-adrian/PSDA/blob/main/01_Exercise/Aufgabe_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aufgabe 4: Korrelation und Multikollinearität

a) Lesen Sie den Datensatz Credit.csv ein. Erkunden Sie die Daten und konvertieren Sie
gegebenenfalls kategorische Variablen in ein geeignetes Format (z. B. mit pandas.get_dummies()
oder sklearn.model_selection.OneHotEncoder()). Die Zielvariable des Datensatz ist Balance.

b) Separieren Sie den Datensatz in 80%-20% für Training und Testdatensatz. Was müssen Sie hier beachten?

c) Erstellen Sie aus der Variable Ethnicity drei binäre Variablen Ethnicity_Caucasian,
Ethnicity_Asian und Ethnicity_African. Führen Sie eine lineare Regression mit OLS als
Schätzer mit diesen Variablen und allen anderen Variablen (außer Limit) durch. Berechnen Sie
Root Mean Square Error (RMSE) zwischen der Vorhersage und den wahren Werten im Test-
Datensatz. Beschreiben Sie Ihre Ergebnisse.

d) Führen Sie die Analyse in 4c) erneut durch, ohne die Variable Ethnicity_Caucasian. Vergleichen
Sie diese Ergebnisse mit denen aus 4c).

e) Führen Sie eine lineare Regression (OLS) mit folgenden Variablen durch:

i. Income, Limit, Age

ii. Income, Rating, Age

iii. Income, Limit, Rating, Age

Beschreiben Sie die Koeffizienten, Konfidenzintervalle und Signifikanzniveau. Vergleichen Sie die
Ergebnisse.

f) Was ist Korrelation? Wann benutzt man Pearson Korrelation, Spearman Korrelation und Kendalls
Tau? Berechnen Sie Korrelationen zwischen allen Variablen im Trainings-Datensatz. Welche
Schlussfolgerung können Sie daraus ziehen?

g) Führen Sie die Analyse in 4e) mit Random Forest anstatt der linearen Regression durch. Variieren
Sie dabei den Parameter random_state mit den Werten {1, 33, 135, 123, 99, 22} und
einigen selbstgewählten Werten. Berechnen Sie MSE des Test-Datensatzes. Vergleichen Sie
Feature Importance. Welche Schlussfolgerung können Sie daraus ziehen?

h) Was ist Multikollinearität? Was sind ihre möglichen Ursachen und Auswirkungen? Wie kann man
sie erkennen? Welche Maßnahmen gibt es, wenn Variablen in den Daten kollinear sind?

a) Lesen Sie den Datensatz Credit.csv ein. Erkunden Sie die Daten und konvertieren Sie gegebenenfalls kategorische Variablen in ein geeignetes Format (z. B. mit pandas.get_dummies() oder sklearn.model_selection.OneHotEncoder()). Die Zielvariable des Datensatz ist Balance.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("drive/My Drive/PSDA/01_Exercise/dataset/Credit.csv",index_col=0)
columns = ['Gender','Student','Married']
for column in columns:
  df[column] = pd.factorize(df[column])[0]
df

,Income,Limit,Rating,Cards,Age,Education,Gender,Student,Married,Ethnicity,Balance
1,14.891,3606,283,2,34,11,0,0,0,Caucasian,333
2,106.025,6645,483,3,82,15,1,1,0,Asian,903
3,104.593,7075,514,4,71,11,0,0,1,Asian,580
4,148.924,9504,681,3,36,11,1,0,1,Asian,964
5,55.882,4897,357,2,68,16,0,0,0,Caucasian,331
...,...,...,...,...,...,...,...,...,...,...,...
396,12.096,4100,307,3,32,13,0,0,0,Caucasian,560
397,13.364,3838,296,5,65,17,0,0,1,African American,480
398,57.872,4171,321,5,67,12,1,0,0,Caucasian,138
399,37.728,2525,192,1,44,13,0,0,0,Caucasian,0


In [4]:
X=df.drop(labels='Balance',axis=1)
y=df['Balance']

b) Separieren Sie den Datensatz in 80%-20% für Training und Testdatensatz. Was müssen Sie hier beachten?

**Antwort:**

Hier gilt zu beachten, dass der Train-Test-Split randomisiert ist. Dies übernimmt die Funktion `train_test_split` von `sklearn` automatisch. Für die Reproduzierbarkeit kann hier ein `random_state` angegeben werden, welcher für die Pseudorandomisierung sorgt.

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=21)
X_train

,Income,Limit,Rating,Cards,Age,Education,Gender,Student,Married,Ethnicity
192,124.290,9560,701,3,52,17,1,1,1,Asian
279,128.040,6982,518,2,78,11,1,0,0,Caucasian
273,30.413,3690,299,2,25,15,1,1,1,Asian
129,123.299,8376,610,2,89,17,0,1,1,African American
286,18.967,1626,156,2,41,11,1,0,0,Asian
...,...,...,...,...,...,...,...,...,...,...
369,89.000,5759,440,3,37,6,1,0,1,Caucasian
49,44.522,2252,205,6,72,15,0,0,0,Asian
261,67.937,5184,383,4,63,12,0,0,0,Asian
313,27.825,5227,386,6,63,11,0,0,0,Caucasian


c) Erstellen Sie aus der Variable Ethnicity drei binäre Variablen Ethnicity_Caucasian, Ethnicity_Asian und Ethnicity_African. Führen Sie eine lineare Regression mit OLS als Schätzer mit diesen Variablen und allen anderen Variablen (außer Limit) durch. Berechnen Sie Root Mean Square Error (RMSE) zwischen der Vorhersage und den wahren Werten im Test- Datensatz. Beschreiben Sie Ihre Ergebnisse.

In [6]:
X_train, X_test = pd.get_dummies(X_train), pd.get_dummies(X_test)
X_train.rename(columns = {'Ethnicity_African American':'Ethnicity_African'}, inplace = True)
X_test.rename(columns = {'Ethnicity_African American':'Ethnicity_African'}, inplace = True)


X_train

,Income,Limit,Rating,Cards,Age,Education,Gender,Student,Married,Ethnicity_African,Ethnicity_Asian,Ethnicity_Caucasian
192,124.290,9560,701,3,52,17,1,1,1,0,1,0
279,128.040,6982,518,2,78,11,1,0,0,0,0,1
273,30.413,3690,299,2,25,15,1,1,1,0,1,0
129,123.299,8376,610,2,89,17,0,1,1,1,0,0
286,18.967,1626,156,2,41,11,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
369,89.000,5759,440,3,37,6,1,0,1,0,0,1
49,44.522,2252,205,6,72,15,0,0,0,0,1,0
261,67.937,5184,383,4,63,12,0,0,0,0,1,0
313,27.825,5227,386,6,63,11,0,0,0,0,0,1


In [7]:
print("Balance")
print("Minimum:", df["Balance"].min())
print("Maximum", df["Balance"].max())
print("Mittelwert:", df["Balance"].mean())

Balance
Minimum: 0
Maximum 1999
Mittelwert: 520.015


In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

reg = LinearRegression().fit(X_train.drop('Limit',axis=1), y_train)
y_pred = reg.predict(X_test.drop('Limit',axis=1))

print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))

RMSE: 103.45462752456523


**Antwort:**

RMSE ist ein Maß für die Abweichung/Error einer Prädiktion. Als RMSE haben wir hier 103.45 (gerundet auf 2 Nachkommastellen), wobei ein niedriger RMSE besser und ein höherer schlechter ist. Beim RMSE ist zu beachten, welche Werte die eingegebenen Variablen haben. Der Zielwert für die Lineare Regression liegt in einem Intervall zwischen 0 und 1999 mit einem Mittelwert von 520.015. Somit ist der RMSE Wert recht hoch, da eine Abweichung von ca. 100 eine große Auswirkung im Ergebnis darstellt.

d) Führen Sie die Analyse in 4c) erneut durch, ohne die Variable Ethnicity_Caucasian. Vergleichen Sie diese Ergebnisse mit denen aus 4c).

In [9]:
reg = LinearRegression().fit(X_train.drop(['Limit','Ethnicity_Caucasian'],axis=1), y_train)
y_pred = reg.predict(X_test.drop(['Limit','Ethnicity_Caucasian'],axis=1))

print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))

RMSE: 103.4546275245652


**Antwort:**

Durch das Löschen von "Ethnicity_Caucasian" gehen keine Informationen verloren, da diese Spalte durch die 2 Spalten "Ethnicity_African" und "Ethnicity_Asian" rekonstruiert werden kann. Somit bleibt der Error (RMSE) unverändert. Die (sehr) geringe Abweichung kommt wahrscheilich durch Ungenauigkeiten der Fließkommadarstellung zustande.

e) Führen Sie eine lineare Regression (OLS) mit folgenden Variablen durch:

i. Income, Limit, Age

ii. Income, Rating, Age

iii. Income, Limit, Rating, Age

Beschreiben Sie die Koeffizienten, Konfidenzintervalle und Signifikanzniveau. Vergleichen Sie die Ergebnisse.

In [10]:
import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


                                 OLS Regression Results                                
Dep. Variable:                Balance   R-squared (uncentered):                   0.925
Model:                            OLS   Adj. R-squared (uncentered):              0.924
Method:                 Least Squares   F-statistic:                              1306.
Date:                Thu, 05 May 2022   Prob (F-statistic):                   5.22e-178
Time:                        18:29:12   Log-Likelihood:                         -2130.3
No. Observations:                 320   AIC:                                      4267.
Df Residuals:                     317   BIC:                                      4278.
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [29]:
model = sm.OLS(y_train, X_train[['Income','Limit','Age']]).fit()
y_pred = model.predict(X_test[['Income','Limit','Age']])

print('RMSE:',mean_squared_error(y_test, y_pred, squared=False))
print(model.summary())

RMSE: 175.28723838126177
                                 OLS Regression Results                                
Dep. Variable:                Balance   R-squared (uncentered):                   0.925
Model:                            OLS   Adj. R-squared (uncentered):              0.924
Method:                 Least Squares   F-statistic:                              1306.
Date:                Thu, 05 May 2022   Prob (F-statistic):                   5.22e-178
Time:                        18:45:55   Log-Likelihood:                         -2130.3
No. Observations:                 320   AIC:                                      4267.
Df Residuals:                     317   BIC:                                      4278.
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------

In [28]:
model = sm.OLS(y_train, X_train[['Income','Rating','Age']]).fit()
y_pred = model.predict(X_test[['Income','Rating','Age']])

print('RMSE:',mean_squared_error(y_test, y_pred, squared=False))
print(model.summary())

RMSE: 193.05468711946858
                                 OLS Regression Results                                
Dep. Variable:                Balance   R-squared (uncentered):                   0.914
Model:                            OLS   Adj. R-squared (uncentered):              0.913
Method:                 Least Squares   F-statistic:                              1127.
Date:                Thu, 05 May 2022   Prob (F-statistic):                   1.04e-168
Time:                        18:45:53   Log-Likelihood:                         -2151.9
No. Observations:                 320   AIC:                                      4310.
Df Residuals:                     317   BIC:                                      4321.
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------

In [27]:
model = sm.OLS(y_train, X_train[['Income','Limit','Rating','Age']]).fit()
y_pred = model.predict(X_test[['Income','Limit', 'Rating', 'Age']])

print('RMSE:',mean_squared_error(y_test, y_pred, squared=False))
print(model.summary())

RMSE: 171.26298247675277
                                 OLS Regression Results                                
Dep. Variable:                Balance   R-squared (uncentered):                   0.926
Model:                            OLS   Adj. R-squared (uncentered):              0.925
Method:                 Least Squares   F-statistic:                              993.2
Date:                Thu, 05 May 2022   Prob (F-statistic):                   1.63e-177
Time:                        18:38:16   Log-Likelihood:                         -2127.7
No. Observations:                 320   AIC:                                      4263.
Df Residuals:                     316   BIC:                                      4278.
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------

**Antwort:**

Das Modell, das auf 'Income','Limit','Rating','Age' trainiert wird hat den geringsten RSME. Dieses Modell hat die meisten Parameter und somit wahrscheinlich mehr relevante Informationen zur verfügung als die anderen Modelle.

Model.Summary gibt uns die Koeffizienten, ein 95% Konvidenzintervall: [0.025 - 0.975] mit dem Signifikanzniveau (P>|t|) für jedes Modell aus. Hierbei Fällt auf, dass das Signifikanzniveau im allgemeineen als p= 0.001 angenommen werden kann bis auf Rating im letzten Modell, hier ist das Signifikanzniveau bei P=0.025.

Die Variable Limit hat im Allgemeinen in beiden Modellen einen sehr geringen Koeffizienten, dies deutet auf einen sehr geringen Einfluss auf die Prädiktion hin.

Fast alle Koffizienten für die Variablen sind negativ. Dies deutet auf eine negative Beziehung zu Balance hin.

Die Koeffizienten und Konfidenzintervalle schwanken zwischen den Modellen.

f) Was ist Korrelation? Wann benutzt man Pearson Korrelation, Spearman Korrelation und Kendalls Tau? Berechnen Sie Korrelationen zwischen allen Variablen im Trainings-Datensatz. Welche Schlussfolgerung können Sie daraus ziehen?

**Antwort:**

1.   Pearson Korrelation: Die Pearson Korrelation wird verwendet um den Lineare Zusammenhang zwischen zwei Variablen zu untersuchen. Die Pearson Korrelation funktioniert am besten mit normalverteilten Daten und nimmt diese als Basis der Berechnung.

2.   Spearman Korrelation: Spearman Korrelation ist eine nicht-parametrische Korrelationsfunktion, die keine Annahmen über die zugrundeliegende Verteilung trifft.

3.   Kendall's Tau: Kendall's Tau vergleicht die Ordnung von Werten in 2 Zufallsvariablen und nicht die Werte selbst.

In [14]:
X_train.corr()

,Income,Limit,Rating,Cards,Age,Education,Gender,Student,Married,Ethnicity_African,Ethnicity_Asian,Ethnicity_Caucasian
Income,1.000000,0.777158,0.775620,-0.051370,0.155110,-0.035646,0.018518,0.061343,-0.051370,0.036428,0.015019,-0.044361
Limit,0.777158,1.000000,0.996662,-0.000916,0.078530,-0.063974,0.021121,0.028888,-0.031501,0.027998,0.012942,-0.035326
Rating,0.775620,0.996662,1.000000,0.043386,0.079590,-0.069158,0.023418,0.037252,-0.041074,0.030316,0.011594,-0.036121
Cards,-0.051370,-0.000916,0.043386,1.000000,0.009853,-0.016079,-0.029440,-0.029978,-0.014234,0.012139,0.010588,-0.019696
Age,0.155110,0.078530,0.079590,0.009853,1.000000,0.017717,-0.015426,0.007208,0.062244,0.070057,-0.036715,-0.027585
Education,-0.035646,-0.063974,-0.069158,-0.016079,0.017717,1.000000,0.017855,0.057574,-0.072656,0.004417,0.035034,-0.034606
Gender,0.018518,0.021121,0.023418,-0.029440,-0.015426,0.017855,1.000000,0.040801,0.014531,-0.039542,0.023982,0.012702
Student,0.061343,0.028888,0.037252,-0.029978,0.007208,0.057574,0.040801,1.000000,0.119390,0.001427,0.054589,-0.049259
Married,-0.051370,-0.031501,-0.041074,-0.014234,0.062244,-0.072656,0.014531,0.119390,1.000000,0.077391,-0.066018,-0.008069
Ethnicity_African,0.036428,0.027998,0.030316,0.012139,0.070057,0.004417,-0.039542,0.001427,0.077391,1.000000,-0.335835,-0.559407


**Antwort:**

Die Korrelationen zwischen den Variablen im Trainingsdatensatz ist im Allgemeinen sehr niedrig (|r| < 0.1). Damit sind die meisten Input-Variablen unabhängig von einander. Eine Ausnahme bilden die Korrelationen zwischen `Income`, `Limit` und `Rating`. Diese sind stark korreliert (|r| > 0.75). Da das Einkommen bestimmt, wie viel Geld man regelmäßig zur Verfügung hat, ist es zu erwarten, dass diese Variable die Variablen `Limit` und `Rating` positiv beeinflusst. Eine weitere Ausnahme bildet die Korrelation zwischen den Variablen `Income` und `Age` (r = 0.16). Diese beiden Variablen scheinen eine leicht Korrelation aufzuweisen. Auch dies lässt sich leicht dadurch erklären, dass die meisten Menschen im Laufe ihres Lebens in den Karrierestufen aufsteigen und somit im Allgemeinen mehr Geld verdienen, je älter sie werden.

Die meisten Korrelationen sind positiv, nur bei den Variablen `Cards`, `Education`, `Married` und `Ethnicity_Caucasian` sind viele negative Korrelationen vorhanden. Generell gibt es bei den Ethnicity-Variablen ein paar negative Korrelationen , welche allerdings sehr klein sind.

g) Führen Sie die Analyse in 4e) mit Random Forest anstatt der linearen Regression durch. Variieren Sie dabei den Parameter random_state mit den Werten {1, 33, 135, 123, 99, 22} und einigen selbstgewählten Werten. Berechnen Sie MSE des Test-Datensatzes. Vergleichen Sie Feature Importance. Welche Schlussfolgerung können Sie daraus ziehen?

In [15]:
from sklearn.ensemble import RandomForestClassifier

In [16]:
values = [1, 33, 135, 123, 99, 22, 500, 12, 49]
for num in values:
  rfc = RandomForestClassifier(random_state=num).fit(X_train, y_train)
  y_pred = rfc.predict(X_test)

  print("random state value:", num)
  print('MSE:',mean_squared_error(y_test, y_pred))
  print("Feature importance:", rfc.feature_importances_)
  print()

random state value: 1
MSE: 59262.0875
Feature importance: [0.13488928 0.20624356 0.19357725 0.07867879 0.12463635 0.10344067
 0.03208576 0.02805326 0.03303446 0.02149156 0.02073392 0.02313514]

random state value: 33
MSE: 73411.1375
Feature importance: [0.13935283 0.20325541 0.20168026 0.07662626 0.12311554 0.09964407
 0.03403384 0.02600827 0.03154304 0.0219106  0.02072999 0.02209988]

random state value: 135
MSE: 68274.825
Feature importance: [0.13799284 0.20312702 0.18710577 0.08043024 0.12963841 0.10435201
 0.0324952  0.02812252 0.0308426  0.0224523  0.0201651  0.02327599]

random state value: 123
MSE: 89521.8
Feature importance: [0.14021432 0.18831691 0.21123915 0.07780538 0.1303928  0.09618015
 0.03256245 0.02819276 0.03182272 0.02058251 0.02134344 0.02134742]

random state value: 99
MSE: 62298.5375
Feature importance: [0.13620676 0.19251906 0.20236336 0.07786282 0.13050144 0.10365139
 0.03232391 0.02550786 0.03252948 0.02128114 0.02261444 0.02263834]

random state value: 22
MSE: 

**Antwort:**

Die Feature Importance bleibt über die verschiedenen Random states relativ konstant. Der MSE variiert je nach Random state sehr stark. Daraus lässt sich interpretieren, dass es für den Random Forest Classifier wichtig ist, in welchen initialen Zustand er startet, um zu einem guten Ergebnis konvergieren zu können. Die Werte der Feature Importance scheinen sich jedoch ähnlichen Werten zu nähern.

h) Was ist Multikollinearität? Was sind ihre möglichen Ursachen und Auswirkungen? Wie kann man
sie erkennen? Welche Maßnahmen gibt es, wenn Variablen in den Daten kollinear sind?

**Antwort:**

Multikollinearität tritt auf, wenn eine erklärende Variable linear durch eine oder mehrere erklärende Variablen mit einer hohen Präzision prädiziert werden kann.

Wenn Multikollinearität vorliegt, dann sind die Koeffizienten des Regressors instabil. Multikollinearität hat aber keine Auswirkungen auf die Präzision der Prädiktion oder die Zuverlässigkeit des Modells.

Multikollinarität lässt sich durch ein hohes Bestimmtheitsmaß zusammen mit niedrigen t-Werten feststellen.

Mögliche Lösungen sind: Variablen entfernen, Hauptkomponentenanalyse oder andere Regressionsverfahren wie Ridge oder Lasso Regression.